# Importações

In [2]:
import os
import zipfile
import numpy as np
import tensorflow as tf
import sys
import argparse
import cv2
import random
import json
# import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras import layers
from sklearn.utils import shuffle
from scipy import ndimage
from tensorflow.python.client import device_lib
# from PIL import Image

# Checa se a GPU está sendo usada

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# Variáveis Globais

In [5]:
train_size        = 185 * 2 # Tamanho dos dados para treinamento
test_size         = 40 # Tamanho dos dados para teste
max_vid_per_class = 20 # Número máximo de vídeos por classe
batch_size        = train_size + test_size # Tamanho total dos dados de treinamento e validação 577
num_of_frames     = 12  # Número de frames que o vídeo será dividido
width             = 64  # Largura da imagem
height            = 48  # Altura da imagem
num_of_channels   = 1   # Número de canais da imagem (cinza, RGB, etc.)
num_of_classes    = 2  # Número de classes que serão utilizadas
test_percentage   = 0.2 # Porcentagem utilizada para a qtd de dados para teste

# Classes (Apenas Vídeo)

In [48]:
class_names = {
    "Acontecer"   : 0, 
    "Aluno"       : 1, 
    "Amarelo"     : 2, 
    'America'     : 3, 
    'Aproveitar'  : 4,
    "Bala"        : 5, 
    "Banco"       : 6, 
    "Banheiro"    : 7, 
    "Barulho"     : 8, 
    "Cinco"       : 9,
    "Conhecer"    : 10, 
    "Espelho"     : 11, 
    "Esquina"     : 12, 
    "Filho"       : 13, 
    "Maca"        : 14, 
    "Medo"        : 15,
    "Ruim"        : 16, 
    "Sapo"        : 17, 
    "Vacina"      : 18, 
    "Vontade"     : 19
}

# Classes (Apenas Imagens)

In [4]:
class_names = {
    "A"   : 0,
    "B"   : 1,
    "C"   : 2,
    "D"   : 3,
    "E"   : 4
}

# Classes (Vídeos e Imagens)

In [14]:
class_names = {
    "A"           : 0,
    "Acontecer"   : 1
}

# Transformação dos vídeos e imagens para treinamento e teste (junto)

In [15]:
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

X_data = [] # armazena os frames
Y_data = [] # armazena as labels

video_type = '.mp4'

angles = [-20, -10, -5, 5, 10, 20] # Define alguns ângulos para rotação

video_classes   = './LIBRAS-Dataset/' # caminho da pasta com os arquivos
list_of_classes = os.listdir(video_classes) # pega o nome de todos os arquivos dentro da pasta
print(list_of_classes)

class_counter = 0
for k in list_of_classes:
    if (class_counter >= num_of_classes):
        break
    
    class_counter = class_counter + 1
    
    X_dummy = []
    Y_dummy = []
    
    video_folder   = str(video_classes + k + '/')
    list_of_videos = os.listdir(video_folder)
    
    contador = 0
    
    print('\n===== ' + k + ' =====\n')
    
    # pega o nome de cada arquivo de vídeo e passa para transformar em frames
    for i in list_of_videos:
        if contador >= max_vid_per_class:
            print('\n\n===== FIM DA CLASSE =====\n')
            break
            
        contador = contador + 1
        
        isVideo = False
        
        if video_type in i:
            isVideo = True
            
        if (isVideo):
            vid = str(video_folder + i) # caminho do vídeo
            cap = cv2.VideoCapture(vid) # lê o vídeo
            
            # pega o tamanho do vídeo em milisegundos e divide pela
            # quantidade de frames para descobrir o tempo que deve ser
            # pego cada frame
            fps          = cap.get(cv2.CAP_PROP_FPS)
            frame_count  = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            duration     = frame_count/fps
            seconds      = frame_count/fps;
            miliseconds  = seconds * 1000
            frame_moment = int(miliseconds/num_of_frames)
        else:
            vid = str(video_folder + i) # caminho do arquivo
            cap = cv2.imread(vid) # lê o arquivo   
        
        frames        = [] # armazenar os frames
        #rotatedFrames = [] # armazenar os frames distorcidos
        
        count = 0 # contador para pegar cada frame
        
        #angle = random.choice(angles) # escolhe aleatóriamente um ângulo
        for j in range(num_of_frames): # aqui pegamos n frames de acordo com num_of_frames
            
            if (isVideo):
                cap.set(cv2.CAP_PROP_POS_MSEC,(count*frame_moment)) # seta o momento do vídeo para pegar o frame
                ret, frame = cap.read() # pega de fato o frame, e se deu sucesso ou não
            
                if ret == True: # se deu sucesso...
                    print('S=>', end="")
                    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # converte o frame para cinza
                    frame = cv2.resize(frame,(width,height),interpolation=cv2.INTER_AREA) # redimensiona o frame
                    frames.append(frame.tolist()) # adiciona o frame para o vetor de frames
                    #rotatedFrame = ndimage.rotate(frame, angle, reshape=False)
                    #rotatedFrames.append(rotatedFrame)
                else:
                    print('*E*=>', end="")
            else:
                print('S=>', end="")

                frame = cv2.cvtColor(cap, cv2.COLOR_BGR2GRAY) # converte o frame para cinza
                frame = cv2.resize(frame,(width,height),interpolation=cv2.INTER_AREA) # redimensiona o frame
                frames.append(frame.tolist()) # adiciona o frame para o vetor de frames
                
            count = count + 1
            
        X_dummy.append(frames) # adiciona todos os frames de um vídeo
        Y_dummy.append(class_names[k]) # adiciona a label do conjunto de frames
    
    X_data.append(X_dummy)
    Y_data.append(Y_dummy)
    #X_data.append(rotatedFrames) # adiciona todos os frames de um vídeo
    #Y_data.append(class_names[k]) # adiciona a label do conjunto de frames
    
print('\n\n===== FIM =====\n\n')

['A', 'Acontecer']

===== A =====

S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>

===== FIM DA CLASSE =====


===== Acontecer =====

S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>

# Transformação das imagens para treinamento e teste (junto)
- Aqui é pego todos os dados juntos, para que a separação entre treino e teste possa ser feita depois

In [5]:
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

X_data = [] # armazena os frames
Y_data = [] # armazena as labels

angles = [-20, -10, -5, 5, 10, 20] # Define alguns ângulos para rotação

video_classes   = './LIBRAS-Dataset-Images/' # caminho da pasta com os arquivos
list_of_classes = os.listdir(video_classes) # pega o nome de todos os arquivos dentro da pasta
print(list_of_classes)

class_counter = 0
for k in list_of_classes:
    if (class_counter >= num_of_classes):
        break
    
    class_counter = class_counter + 1
    
    X_dummy = []
    Y_dummy = []
    
    video_folder   = str(video_classes + k + '/')
    list_of_videos = os.listdir(video_folder)
    
    contador = 0
    
    print('\n===== ' + k + ' =====\n')
    
    # pega o nome de cada arquivo de vídeo e passa para transformar em frames
    for i in list_of_videos:
        if contador >= max_vid_per_class:
            print('\n\n===== FIM DA CLASSE =====\n')
            break
            
        contador = contador + 1
        
        vid = str(video_folder + i) # caminho do vídeo
        cap = cv2.imread(vid) # lê o vídeo
        
        # pega o tamanho do vídeo em milisegundos e divide pela
        # quantidade de frames para descobrir o tempo que deve ser
        # pego cada frame
#         fps          = cap.get(cv2.CAP_PROP_FPS)
#         frame_count  = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
#         duration     = frame_count/fps
#         seconds      = frame_count/fps;
#         miliseconds  = seconds * 1000
#         frame_moment = int(miliseconds/num_of_frames)
        
        frames        = [] # armazenar os frames
#         rotatedFrames = [] # armazenar os frames distorcidos
        
        count = 0 # contador para pegar cada frame
        
        angle = random.choice(angles) # escolhe aleatóriamente um ângulo
        for j in range(num_of_frames): # aqui pegamos n frames de acordo com num_of_frames
#             cap.set(cv2.CAP_PROP_POS_MSEC,(count*frame_moment)) # seta o momento do vídeo para pegar o frame
#             ret, frame = cap.read() # pega de fato o frame, e se deu sucesso ou não
            
#             if ret == True: # se deu sucesso...
            print('S=>', end="")

            frame = cv2.cvtColor(cap, cv2.COLOR_BGR2GRAY) # converte o frame para cinza
            frame = cv2.resize(frame,(width,height),interpolation=cv2.INTER_AREA) # redimensiona o frame
            frames.append(frame.tolist()) # adiciona o frame para o vetor de frames
            #rotatedFrame = ndimage.rotate(frame, angle, reshape=False)
            #rotatedFrames.append(rotatedFrame)
                
#             else:
#                 print('*E*=>', end="")
                
            count = count + 1
            
        X_dummy.append(frames) # adiciona todos os frames de um vídeo
        Y_dummy.append(class_names[k]) # adiciona a label do conjunto de frames
    
    X_data.append(X_dummy)
    Y_data.append(Y_dummy)
    #X_data.append(rotatedFrames) # adiciona todos os frames de um vídeo
    #Y_data.append(class_names[k]) # adiciona a label do conjunto de frames
    
print('\n\n===== FIM =====\n\n')

['A', 'B', 'C', 'D', 'E']

===== A =====

S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S

# Transformação dos frames de vídeos para treinamento e teste (junto)
- Aqui é pego todos os dados juntos, para que a separação entre treino e teste possa ser feita depois

In [8]:
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

X_data = [] # armazena os frames
Y_data = [] # armazena as labels

angles = [-20, -10, -5, 5, 10, 20] # Define alguns ângulos para rotação

video_classes   = './LIBRAS-Dataset/Treino/' # caminho da pasta com os arquivos
list_of_classes = os.listdir(video_classes) # pega o nome de todos os arquivos dentro da pasta
print(list_of_classes)

class_counter = 0
for k in list_of_classes:
    if (class_counter >= num_of_classes):
        break
    
    class_counter = class_counter + 1
    
    X_dummy = []
    Y_dummy = []
    
    video_folder   = str(video_classes + k + '/')
    list_of_videos = os.listdir(video_folder)
    
    contador = 0
    
    print('\n===== ' + k + ' =====\n')
    
    # pega o nome de cada arquivo de vídeo e passa para transformar em frames
    for i in list_of_videos:
        if contador >= max_vid_per_class:
            print('\n\n===== FIM DA CLASSE =====\n')
            break
            
        contador = contador + 1
        
        vid = str(video_folder + i) # caminho do vídeo
        cap = cv2.VideoCapture(vid) # lê o vídeo
        
        # pega o tamanho do vídeo em milisegundos e divide pela
        # quantidade de frames para descobrir o tempo que deve ser
        # pego cada frame
        fps          = cap.get(cv2.CAP_PROP_FPS)
        frame_count  = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        duration     = frame_count/fps
        seconds      = frame_count/fps;
        miliseconds  = seconds * 1000
        frame_moment = int(miliseconds/num_of_frames)
        
        frames        = [] # armazenar os frames
        rotatedFrames = [] # armazenar os frames distorcidos
        
        count = 0 # contador para pegar cada frame
        
        angle = random.choice(angles) # escolhe aleatóriamente um ângulo
        for j in range(num_of_frames): # aqui pegamos n frames de acordo com num_of_frames
            cap.set(cv2.CAP_PROP_POS_MSEC,(count*frame_moment)) # seta o momento do vídeo para pegar o frame
            ret, frame = cap.read() # pega de fato o frame, e se deu sucesso ou não
            
            if ret == True: # se deu sucesso...
                print('S=>', end="")
                
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # converte o frame para cinza
                frame = cv2.resize(frame,(width,height),interpolation=cv2.INTER_AREA) # redimensiona o frame
                frames.append(frame.tolist()) # adiciona o frame para o vetor de frames
                #rotatedFrame = ndimage.rotate(frame, angle, reshape=False)
                #rotatedFrames.append(rotatedFrame)
                
            else:
                print('*E*=>', end="")
                
            count = count + 1
            
        X_dummy.append(frames) # adiciona todos os frames de um vídeo
        Y_dummy.append(class_names[k]) # adiciona a label do conjunto de frames
    
    X_data.append(X_dummy)
    Y_data.append(Y_dummy)
    #X_data.append(rotatedFrames) # adiciona todos os frames de um vídeo
    #Y_data.append(class_names[k]) # adiciona a label do conjunto de frames
    
print('\n\n===== FIM =====\n\n')

FileNotFoundError: [WinError 3] O sistema não pode encontrar o caminho especificado: './LIBRAS-Dataset/Treino/'

# Transformação dos frames de vídeos para treino (separado)

In [64]:
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

X_data_train = [] # armazena os frames
Y_data_train = [] # armazena as labels
angles = [-20, -10, -5, 5, 10, 20] # Define alguns ângulos para rotação

video_classes   = './LIBRAS-Dataset-2/Treino/' # caminho da pasta com os arquivos
list_of_classes = os.listdir(video_classes) # pega o nome de todos os arquivos dentro da pasta
print(list_of_classes)

class_counter = 0
for k in list_of_classes:
    if (class_counter >= num_of_classes):
        break
        
    class_counter = class_counter + 1
    
    video_folder   = str(video_classes + k + '/')
    list_of_videos = os.listdir(video_folder)
    
    contador = 0
    
    print('\n===== ' + k + ' =====\n')
    
    # pega o nome de cada arquivo de vídeo e passa para transformar em frames
    for i in list_of_videos:
        if contador >= max_vid_per_class:
            print('\n\n===== FIM DA CLASSE =====\n')
            break
            
        contador = contador + 1
        
        vid = str(video_folder + i) # caminho do vídeo
        cap = cv2.VideoCapture(vid) # lê o vídeo
        
        # pega o tamanho do vídeo em milisegundos e divide pela
        # quantidade de frames para descobrir o tempo que deve ser
        # pego cada frame
        fps          = cap.get(cv2.CAP_PROP_FPS)
        frame_count  = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        duration     = frame_count/fps
        seconds      = frame_count/fps;
        miliseconds  = seconds * 1000
        frame_moment = int(miliseconds/num_of_frames)
        
        frames        = [] # armazenar os frames
        rotatedFrames = [] # armazenar os frames distorcidos
        
        count = 0 # contador para pegar cada frame
        
        angle = random.choice(angles) # escolhe aleatóriamente um ângulo
        
        for j in range(num_of_frames): # aqui pegamos n frames de acordo com num_of_frames
            cap.set(cv2.CAP_PROP_POS_MSEC,(count*frame_moment)) # seta o momento do vídeo para pegar o frame
            ret, frame = cap.read() # pega de fato o frame, e se deu sucesso ou não
            
            if ret == True: # se deu sucesso...
                print('S=>', end="")
                
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # converte o frame para cinza
                frame = cv2.resize(frame,(width,height),interpolation=cv2.INTER_AREA) # redimensiona o frame
                frames.append(frame.tolist()) # adiciona o frame para o vetor de frames
                rotatedFrame = ndimage.rotate(frame, angle, reshape=False)
                rotatedFrames.append(rotatedFrame.tolist())
                
            else:
                print('*E*=>', end="")
                
            count = count + 1
            
        X_data_train.append(frames) # adiciona todos os frames de um vídeo
        Y_data_train.append(class_names[k]) # adiciona a label do conjunto de frames
        X_data_train.append(rotatedFrames) # adiciona todos os frames de um vídeo
        Y_data_train.append(class_names[k]) # adiciona a label do conjunto de frames
        
print('\n\n===== FIM =====\n\n')

['Acontecer', 'Aluno', 'Amarelo', 'America', 'Aproveitar', 'Bala', 'Banco', 'Banheiro', 'Barulho', 'Cinco', 'Conhecer', 'Espelho', 'Esquina', 'Filho', 'Maca', 'Medo', 'Ruim', 'Sapo', 'Vacina', 'Vontade']

===== Acontecer =====

S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S

S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S

S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S

# Transformação dos frames de vídeos para teste (separado)

In [66]:
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

X_data_test = [] # armazena os frames
Y_data_test = [] # armazena as labels

video_classes   = './LIBRAS-Dataset-2/Teste/' # caminho da pasta com os arquivos
list_of_classes = os.listdir(video_classes) # pega o nome de todos os arquivos dentro da pasta
print(list_of_classes)

class_counter = 0

for k in list_of_classes:
    if (class_counter >= num_of_classes):
        break
        
    class_counter = class_counter + 1
    
    video_folder   = str(video_classes + k + '/')
    list_of_videos = os.listdir(video_folder)
    
    contador = 0
    
    print('\n===== ' + k + ' =====\n')
    
    # pega o nome de cada arquivo de vídeo e passa para transformar em frames
    for i in list_of_videos:
        if contador >= max_vid_per_class:
            print('\n\n===== FIM DA CLASSE =====\n')
            break
            
        contador = contador + 1
        
        vid = str(video_folder + i) # caminho do vídeo
        cap = cv2.VideoCapture(vid) # lê o vídeo
        
        frames = [] # armazenar os frames
        
        count = 0 # contador para pegar cada frame
        
        for j in range(num_of_frames): # aqui pegamos n frames de acordo com num_of_frames
            cap.set(cv2.CAP_PROP_POS_MSEC,(count*250)) # seta o momento do vídeo para pegar o frame
            ret, frame = cap.read() # pega de fato o frame, e se deu sucesso ou não
            
            if ret == True: # se deu sucesso...
                print('S=>', end="")
                
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # converte o frame para cinza
                frame = cv2.resize(frame,(width,height),interpolation=cv2.INTER_AREA) # redimensiona o frame
                frames.append(frame.tolist()) # adiciona o frame para o vetor de frames
                
            else:
                print('*E*=>', end="")
                
            count = count + 1
            
        X_data_test.append(frames) # adiciona todos os frames de um vídeo
        Y_data_test.append(class_names[k]) # adiciona a label do conjunto de frames

print('\n\n===== FIM =====\n\n')

['Acontecer', 'Aluno', 'Amarelo', 'America', 'Aproveitar', 'Bala', 'Banco', 'Banheiro', 'Barulho', 'Cinco', 'Conhecer', 'Espelho', 'Esquina', 'Filho', 'Maca', 'Medo', 'Ruim', 'Sapo', 'Vacina', 'Vontade']

===== Acontecer =====

S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>
===== Aluno =====

S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>S=>
===== Amarelo =====

S=>S=>S=>S

# Checar se vetor é instância de numpy

In [ ]:
print(isinstance(X_data[0][0][0], np.ndarray))

# Salvar os Dados em JSON (junto)

In [27]:
with open('./video-classes/X_data.json', 'w') as f:
    json.dump(X_data, f)

with open('./video-classes/Y_data.json', 'w') as f:
    json.dump(Y_data, f)

False


# Carregar os Dados JSON (junto)

In [7]:
f = open('./video-classes/X_data.json')
X_data_loaded = json.load(f)

f = open('./video-classes/Y_data.json')
Y_data_loaded = json.load(f)

# Transferir os dados carregados com o número de classes determinada (junto)

In [8]:
X_data = X_data_loaded[:num_of_classes]
Y_data = Y_data_loaded[:num_of_classes]
print('Quantidade de classes => ' + str(len(Y_data)))

10


In [69]:
print(len(X_data_train))
print(isinstance(X_data, np.ndarray))

2096
False


# Salvar os dados em JSON (separado)

In [68]:
with open('./video-classes-2/X_data_train.json', 'w') as f:
    json.dump(X_data_train, f)

with open('./video-classes-2/Y_data_train.json', 'w') as f:
    json.dump(Y_data_train, f)

with open('./video-classes-2/X_data_test.json', 'w') as f:
    json.dump(X_data_test, f)

with open('./video-classes-2/Y_data_test.json', 'w') as f:
    json.dump(Y_data_test, f)

# Carregar os dados JSON (separado)

In [71]:
f = open('./video-classes-2/X_data_train.json')
X_data_train_loaded = json.load(f)

f = open('./video-classes-2/Y_data_train.json')
Y_data_train_loaded = json.load(f)

f = open('./video-classes-2/X_data_test.json')
X_data_test_loaded = json.load(f)

f = open('./video-classes-2/Y_data_test.json')
Y_data_test_loaded = json.load(f)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19]


# Transferir os dados carregados com o número de classes determinada (separado)

In [81]:
X_data_train = np.array(X_data_train_loaded[:train_size])
Y_data_train = np.array(Y_data_train_loaded[:train_size])
X_data_test  = np.array(X_data_test_loaded[:test_size])
Y_data_test  = np.array(Y_data_test_loaded[:test_size])

print('Quantidade de vídeos p/ treino => ' + str(len(Y_data_train)))
print('Quantidade de vídeos p/ teste => ' + str(len(Y_data_test)))

Quantidade de vídeos p/ treino => 370
Quantidade de vídeos p/ teste => 40


# Formato dos dados

In [16]:
print(X_data_train.shape)
print(Y_data_train.shape)
print(Y_data_train)

print(X_data_test.shape)
print(Y_data_test.shape)
print(Y_data_test)

NameError: name 'X_data_train' is not defined

# Embaralhamento dos dados (junto)

In [19]:
X_data_train = []
Y_data_train = []
X_data_test = []
Y_data_test = []

for i in range(num_of_classes):
    X_aux = X_data[i]
    Y_aux = Y_data[i]
    X_aux, Y_aux = shuffle(X_aux, Y_aux)
    
    ratio = int(len(X_aux) * test_percentage)
    print('ratio => ' + str(ratio))
    
    if (len(X_data_train) == 0):
        X_data_train = X_aux[ratio:]
        Y_data_train = Y_aux[ratio:]
        X_data_test  = X_aux[:ratio]
        Y_data_test  = Y_aux[:ratio]
        
    else:    
        X_data_train = np.concatenate((X_data_train, X_aux[ratio:]), axis=0)
        Y_data_train = np.concatenate((Y_data_train, Y_aux[ratio:]), axis=0)
        X_data_test  = np.concatenate((X_data_test, X_aux[:ratio]), axis=0)
        Y_data_test  = np.concatenate((Y_data_test, Y_aux[:ratio]), axis=0)

print(X_data_train.shape)
print(X_data_test.shape)
train_size = X_data_train.shape[0]
test_size = X_data_test.shape[0]
X_data_train = X_data_train.reshape(train_size, num_of_frames, width, height, num_of_channels)
print(np.shape(X_data_train))

X_data_test = X_data_test.reshape(test_size, num_of_frames, width, height, num_of_channels)
print(np.shape(X_data_test))

ratio => 4
ratio => 4
(32, 12, 48, 64)
(8, 12, 48, 64)
(32, 12, 64, 48, 1)
(8, 12, 64, 48, 1)


# Embaralhamento dos dados (separado)

In [ ]:
# X_data, Y_data = shuffle(X_data, Y_data)
# print(Y_data)
# print(X_data.shape[0])
# print(X_data[0].shape)
# print(X_data[1].shape)

# X_data_train = X_data[173:]
# Y_data_train = Y_data[173:]
# X_data_test = X_data[:173]
# Y_data_test = Y_data[:173]
# print(Y_data_train)
# print(Y_data_test)

# print(X_data_train.shape)
# print(X_data_test.shape)

# X_data_train = X_data_train.reshape(train_size, num_of_frames, width, height, num_of_channels)
# print(np.shape(X_data_train))

# X_data_test = X_data_test.reshape(test_size, num_of_frames, width, height, num_of_channels)
# print(np.shape(X_data_test))

# Exemplo de frames de um vídeo

In [18]:
from PIL import Image
import matplotlib.pyplot as plt

# converte a imagem de numpy para Image
image0 = Image.fromarray(X_data_train[0][0])
image1 = Image.fromarray(X_data_train[0][1])
image2 = Image.fromarray(X_data_train[0][2])
image3 = Image.fromarray(X_data_train[0][3])
image4 = Image.fromarray(X_data_train[0][4])
image5 = Image.fromarray(X_data_train[0][5])

# plota as imagens
plt.imshow(image0, 'gray')
f, axarr = plt.subplots(1, 6)
axarr[0].imshow(image0, 'gray')
axarr[1].imshow(image1, 'gray')
axarr[2].imshow(image2, 'gray')
axarr[3].imshow(image3, 'gray')
axarr[4].imshow(image4, 'gray')
axarr[5].imshow(image5, 'gray')

ModuleNotFoundError: No module named 'PIL'

# Remodelar o conjunto de dados
Exemplo:
So, if you have 10000 samples in total, using sets of 10 frames per input, with 30 x 30 dimensions and 1 channel of color, you may reshape your X_data like:

X_data = X_data.reshape(10000, 10, 30, 30, 1)

In [82]:
# transforma os dados dos frames armazenados no formato ideal para ser usado no treinamento
X_data_train = X_data_train.reshape(train_size, num_of_frames, width, height, num_of_channels)
print(np.shape(X_data_train))

X_data_test = X_data_test.reshape(test_size, num_of_frames, width, height, num_of_channels)
print(np.shape(X_data_test))

(370, 12, 64, 48, 1)
(40, 12, 64, 48, 1)


# Definição da Rede Neural Convolucional 3D 

In [6]:
 def get_model():
    
    # Camadas do modelo
    
    model = keras.Sequential()

    model.add(keras.layers.Conv3D(50, (5,7,7), activation='relu', input_shape=(9, width, height, num_of_channels)))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.MaxPooling3D((1, 2, 2), padding='same'))

    model.add(keras.layers.Conv3D(50, (5,7,7), activation='relu'))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.MaxPooling3D((1, 2, 2), padding='same'))
    
    model.add(keras.layers.Conv3D(10, (3,5,5), activation='relu'))
    model.add(keras.layers.BatchNormalization())
    
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(200, activation='sigmoid'))
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Dense(num_of_classes, activation='sigmoid', use_bias=True))
    
    # Compilação do modelo
    
    initial_learning_rate = 0.0001
    lr_schedule = keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True
    )
    
    model.compile(
        loss=keras.losses.SparseCategoricalCrossentropy(),
        optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
        metrics=["acc"],
    )
    
#     model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

    return model

# Constrói o modelo.
model = get_model()

# Dados do modelo
model.summary()

ValueError: Negative dimension size caused by subtracting 3 from 1 for '{{node conv3d_2/Conv3D}} = Conv3D[T=DT_FLOAT, data_format="NDHWC", dilations=[1, 1, 1, 1, 1], padding="VALID", strides=[1, 1, 1, 1, 1]](Placeholder, conv3d_2/Conv3D/ReadVariableOp)' with input shapes: [?,1,12,8,50], [3,5,5,50,10].

# Treinamento do Modelo

In [ ]:
# epochs = 100
# model.fit(X_data_train, Y_data_train, epochs=epochs, batch_size=train_size, shuffle=True)

# Treinamento do modelo até determinada acurácia (separado)

In [ ]:
epochs = 100
accuracy = 0
while accuracy < 0.6:
    model.fit(X_data_train, Y_data_train, epochs=epochs, batch_size=train_size, shuffle=True)
    result = model.evaluate(X_data_test, Y_data_test)
    if (result[1] > accuracy):
        model.save('./model-2/')
        accuracy = result[1]
    print(result[1])

Epoch 1/100


# Treinamento do modelo até determinada acurácia, com dados de entrada aleatórios

In [23]:
epochs = 100
accuracy = 0
while accuracy < 0.6:
    X_data_train = []
    Y_data_train = []
    X_data_test = []
    Y_data_test = []

    for i in range(num_of_classes):
        X_aux = X_data[i]
        Y_aux = Y_data[i]
        X_aux, Y_aux = shuffle(X_aux, Y_aux)

        ratio = int(len(X_aux) * 0.2)

        if (len(X_data_train) == 0):
            X_data_train = X_aux[ratio:]
            Y_data_train = Y_aux[ratio:]
            X_data_test  = X_aux[:ratio]
            Y_data_test  = Y_aux[:ratio]
        else:    
            X_data_train = np.concatenate((X_data_train, X_aux[ratio:]), axis=0)
            Y_data_train = np.concatenate((Y_data_train, Y_aux[ratio:]), axis=0)
            X_data_test  = np.concatenate((X_data_test, X_aux[:ratio]), axis=0)
            Y_data_test  = np.concatenate((Y_data_test, Y_aux[:ratio]), axis=0)

    train_size = X_data_train.shape[0]
    test_size  = X_data_test.shape[0]
    
    X_data_train = X_data_train.reshape(train_size, num_of_frames, width, height, num_of_channels)
    X_data_test  = X_data_test.reshape(test_size, num_of_frames, width, height, num_of_channels)

    model.fit(X_data_train, Y_data_train, epochs=epochs, batch_size=train_size, shuffle=True)
    result = model.evaluate(X_data_test, Y_data_test)
    if (result[1] > accuracy):
        model.save('./model/')
        accuracy = result[1]
    print(result[1])

Epoch 1/100
1/1 [==============================] - 2s 2s/step - loss: 0.8722 - acc: 0.4688
Epoch 2/100
1/1 [==============================] - 1s 905ms/step - loss: 0.6616 - acc: 0.6562
Epoch 3/100
1/1 [==============================] - 1s 912ms/step - loss: 0.4235 - acc: 0.7812
Epoch 4/100
1/1 [==============================] - 1s 909ms/step - loss: 0.3854 - acc: 0.7812
Epoch 5/100
1/1 [==============================] - 1s 909ms/step - loss: 0.3271 - acc: 0.8750
Epoch 6/100
1/1 [==============================] - 1s 904ms/step - loss: 0.2604 - acc: 0.8750
Epoch 7/100
1/1 [==============================] - 1s 910ms/step - loss: 0.2456 - acc: 0.9062
Epoch 8/100
1/1 [==============================] - 1s 910ms/step - loss: 0.1879 - acc: 0.9375
Epoch 9/100
1/1 [==============================] - 1s 911ms/step - loss: 0.1581 - acc: 0.9688
Epoch 10/100
1/1 [==============================] - 1s 903ms/step - loss: 0.1582 - acc: 0.9688
Epoch 11/100
1/1 [==============================] - 1s 908ms/s

# Validação do modelo

In [13]:
model.evaluate(X_data_test, Y_data_test)

1/1 [==============================] - 0s 133ms/step - loss: 0.3342 - acc: 0.9167


[0.33415135741233826, 0.9166666865348816]

# Predição do modelo

In [12]:
prediction = model.predict(np.expand_dims(X_data_test[0], axis=0))[0]
print(Y_data_test)
for predict, name in zip(prediction, class_names):
    print(
        "%.2f ==> %s"
        % ((100 * predict), name)
    )

[0 0 0 0 0 0 1 1 1 1 1 1]
58.24 ==> A
18.79 ==> B


# Salvar o modelo

In [ ]:
model.save('./model/')

# Carregar o modelo

In [20]:
newModel = keras.models.load_model('./model/')

In [21]:
newModel.evaluate(X_data_test, Y_data_test)

6/6 [==============================] - 0s 23ms/step - loss: 0.9130 - acc: 0.9765


[0.9129856824874878, 0.9764705896377563]